<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Setup

**Description:** This notebook allows you to setup abi on local or on your Naas server.

## Input

### Install requirements
Install the packages required to run the product.

In [ ]:
from ipykernel import get_connection_file
import json

kernel_name = json.loads(open(get_connection_file()).read()).get('kernel_name')
if kernel_name != 'abi':
    print('🔴 This notebook is not running the ABI kernel.')
    print('Installing kernel...')
    !make conda-install-kernel
    raise Exception('🟡 Please restart the notebook selecting the "abi" kernel.')
else:
    print('✅ This notebook is running the ABI kernel. Proceeding...')

### Import libraries

In [ ]:
import naas_python
import os
import shutil

### Setup variables
**Pre-requisite:**
- Create an account on [Naas.ai](https://naas.ai/auth/signin) if you don't already have one.

In [ ]:
config_secret_path = 'config.secret.yml'
config_secret_template_path = 'config.secret.template.yml'

## Model

### Get Naas token

In [ ]:
naas_api_token = naas_python.utils.domains_base.authorization.NaasSpaceAuthenticatorAdapter().jwt_token()

## Output

### Create config.secret.yml and update content

In [ ]:
if not os.path.exists(config_secret_path):
    shutil.copy(config_secret_template_path, config_secret_path)
    print(f'✅ {config_secret_path} sucessfully created')
else:
    print(f'🟡 {config_secret_path} already exists.')